In [1]:
import pandas as pd
from datetime import date, timedelta

In [2]:
file_path = "https://www.dhhs.vic.gov.au/ncov-covid-cases-by-age-group-csv"
df = pd.read_csv(file_path)

In [3]:
df.tail()

,diagnosis_date,agegroup
717265,2022-03-22,40-49
717266,2022-03-22,40-49
717267,2022-03-22,0-9
717268,2022-03-22,40-49
717269,2022-03-22,20-29


In [4]:
df_pivot = pd.pivot_table(df, index = 'diagnosis_date', columns = 'agegroup', 
                         values = 'agegroup', aggfunc = len, fill_value = 0)

In [5]:
df_pivot.tail()

agegroup,0-9,10-19_,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+
diagnosis_date,,,,,,,,,,
2022-03-18,288,389,581,520,457,306,200,123,46,18
2022-03-19,225,333,419,425,388,261,194,110,44,16
2022-03-20,220,346,333,362,319,226,150,90,34,5
2022-03-21,241,354,376,448,399,295,194,117,47,18
2022-03-22,313,498,593,624,501,387,257,152,63,17


In [6]:
# check that the latest date's sum matches the number of cases reported by Vic govt
# the latest row's date is always yesterday

yesterday = date.today() - timedelta(days = 1)

# the dates in the pivot table are strings
# so use isoformat() to convert date into string before looking it up in the table
df_pivot.loc[yesterday.isoformat()].sum()

3405

In [7]:
# take only the 0-9 and 10-19 cases, from 5 Dec onwards
# for each date, store the previous date's cases; this matches what I've done for NSW
# eg. on 8 Dec, the Vic cases are those reported on the 7th

first = date(2021, 12, 5)
today = date.today()

df_0_19 = pd.DataFrame(columns = ['date', 'cases_0_9', 'cases_10_19'])

curr = first
prev = first - timedelta(days = 1)

while (curr < today):
    
        df_0_19 = df_0_19.append({'date': curr.isoformat(), 'cases_0_9': df_pivot.loc[prev.isoformat(), '0-9'], 
                                  'cases_10_19': df_pivot.loc[prev.isoformat(), '10-19_']}, ignore_index = True)
        prev = curr
        curr = curr + timedelta(days = 1)

C:\Users\amanj\AppData\Local\Temp\ipykernel_6720\1496215999.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_0_19 = df_0_19.append({'date': curr.isoformat(), 'cases_0_9': df_pivot.loc[prev.isoformat(), '0-9'],
C:\Users\amanj\AppData\Local\Temp\ipykernel_6720\1496215999.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_0_19 = df_0_19.append({'date': curr.isoformat(), 'cases_0_9': df_pivot.loc[prev.isoformat(), '0-9'],
C:\Users\amanj\AppData\Local\Temp\ipykernel_6720\1496215999.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_0_19 = df_0_19.append({'date': curr.isoformat(), 'cases_0_9': df_pivot.loc[prev.isoformat(), '0-9'],
C:\Users\amanj\AppData\Local\Temp\ipykernel_6720\1496215999.py:15: FutureWarning: 

In [8]:
df_0_19.tail()

,date,cases_0_9,cases_10_19
103,2022-03-18,315,493
104,2022-03-19,288,389
105,2022-03-20,225,333
106,2022-03-21,220,346
107,2022-03-22,241,354


In [9]:
df_0_19.to_csv("../data/cases_0_19.csv")